## 0 Import Modules

In [1]:
import os

In [2]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import wandb

In [3]:
from tqdm import tqdm

In [4]:
from neumeta.models import create_densenet_model as create_model
from neumeta.utils import (
    parse_args, print_omegaconf,
    load_checkpoint, save_checkpoint,
    set_seed,
    AverageMeter, 
)

## 1 Functions

### Accuracy

In [5]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

### Training Loop Function

In [6]:
# Function to train the model for one epoch
def train_one_epoch(model, train_loader, optimizer, criterion, epoch_idx, args=None, device='cpu'):
    # Set the model to training mode
    model.train()

    # Initialize AverageMeter objects to track the losses
    losses = AverageMeter()
    top1 = AverageMeter()
    
    # Iterate over the training data
    for batch_idx, (x, target) in enumerate(train_loader):
        # Preprocess input
        # ------------------------------------------------------------------------------------------------------
        # Move the data to the device
        x, target = x.to(device), target.to(device)
        
        # Compute output and loss
        predict = model(x)
        loss = criterion(predict, target) 
        
        # Measure accuracy and record loss
        prec1 = accuracy(predict.data, target, topk=(1,))[0].item()
        losses.update(loss.item(), x.size(0))
        top1.update(prec1, x.size(0))

        # Compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss = losses.avg
        train_acc = top1.avg

        # Log (or plot) losses
        # ------------------------------------------------------------------------------------------------------
        # Log the losses and learning rate to wandb
        if batch_idx % args.experiment.log_interval == 0:
            wandb.log({
                "Training Loss": train_loss,
                "Training Accuracy": train_acc
            }, step=batch_idx + epoch_idx * len(train_loader))
            # Print the losses and learning rate
            print(
                f"Iteration {batch_idx}: Loss = {train_loss:.4f}, Top-1 Accuracy = {train_acc:.4f}, Learning rate = {optimizer.param_groups[0]['lr']:.4e}")
    
    return train_loss

### 1.2 Validation function

In [7]:
def validate(model, val_loader, criterion, args=None, device='cpu'):
    # Set the model to training mode
    model.eval()

    # Initialize AverageMeter objects to track the losses
    losses = AverageMeter()
    top1 = AverageMeter()
    
    with torch.no_grad():
        # Iterate over the training data
        for x, target in tqdm(val_loader):
            # Preprocess input
            x, target = x.to(device), target.to(device)
            predict = model(x)
            
            loss = criterion(predict, target) 
            
            # Measure accuracy and record loss
            prec1 = accuracy(predict.data, target, topk=(1,))[0].item()
            losses.update(loss.item(), x.size(0))
            top1.update(prec1, x.size(0))

    return losses.avg, top1.avg

### 1.4 Adjust learning rate

In [8]:
def adjust_learning_rate(optimizer, epoch, init_lr):
    """Sets the learning rate to the initial LR decayed by 10 after 150 and 225 epochs"""
    if epoch < 150:
        lr = init_lr
    elif epoch < 225:
        lr = init_lr * 0.1
    else:
        lr = init_lr * 0.01
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### 1.5 Initialize wandb

In [9]:
def initialize_wandb(config):
    import time
    """
    Initializes Weights and Biases (wandb) with the given configuration.
    
    Args:
        configuration (dict): Configuration parameters for the run.
    """
    # Name the run using current time and configuration name
    run_name = f"{time.strftime('%Y%m%d%H%M%S')}-{config.experiment.name}"
    
    wandb.init(project="densenet-trial", name=run_name, config=dict(config), group='densenet')

## 2 Training LeNet-dim_32

### 2.0 Device

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Parsing arguments for input

In [14]:
CONFIG_PATH = 'neumeta/config/densenet/densenet_baseline.yaml'
RATIO = '1.0'
CHECKPOINT_PATH = 'toy/experiments/densenet_bc_40_12_baseline/densenet_bc_40_12_cifar10_baseline_best.pth'

In [15]:
argv_train = ['--config', CONFIG_PATH, '--ratio', RATIO, '--resume_from', CHECKPOINT_PATH]
argv_train

['--config',
 'neumeta/config/densenet/densenet_baseline.yaml',
 '--ratio',
 '1.0',
 '--resume_from',
 'toy/experiments/densenet_bc_40_12_baseline/densenet_bc_40_12_cifar10_baseline_best.pth']

In [16]:
args = parse_args(argv_train)
print_omegaconf(args)

+--------------------------+----------------------------------------------------------------------------------------+
|           Key            |                                         Value                                          |
+--------------------------+----------------------------------------------------------------------------------------+
|     experiment.name      |                               densenet_bc_40_12_baseline                               |
|  experiment.num_epochs   |                                          300                                           |
| experiment.log_interval  |                                          100                                           |
| experiment.eval_interval |                                           1                                            |
|     experiment.seed      |                                           42                                           |
|        model.type        |                            

In [17]:
set_seed(args.experiment.seed)

Setting seed... 42 for reproducibility


### 2.2 Data Preparations

In [18]:
normalize = transforms.Normalize(
    mean=[x/255.0 for x in [125.3, 123.0, 113.9]], 
    std=[x/255.0 for x in [63.0, 62.1, 66.7]]
    )

transforms_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

In [19]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transforms_train, download=True)
val_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=args.training.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=args.training.batch_size, shuffle=False)

### 2.3 Create Model

In [21]:
model = create_model(args.model.type,
                     layers=args.model.layers,
                     growth=args.model.growth,
                     compression=args.model.compression,
                     bottleneck=args.model.bottleneck,
                     drop_rate=args.model.drop_rate).to(device)

In [22]:
model

DenseNet3(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): DenseBlock(
    (layer): Sequential(
      (0): BottleneckBlock(
        (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv1): Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (1): BottleneckBlock(
        (bn1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv1): Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(48, 12, kern

In [23]:
for i, k in enumerate(model.parameters()):
    print(k)

Parameter containing:
tensor([[[[-7.6929e-02,  3.8479e-02,  1.1810e-01],
          [ 3.7327e-02, -2.5631e-01,  5.8351e-03],
          [-3.9329e-02, -5.9862e-02, -6.0881e-02]],

         [[-1.1163e-01,  1.0882e-01,  9.3454e-02],
          [-1.1987e-03,  3.1157e-02,  1.7140e-01],
          [ 8.1337e-02, -7.8931e-02,  4.9267e-02]],

         [[ 1.5046e-02, -8.8851e-02,  3.3003e-01],
          [-4.2694e-02, -1.0119e-01,  9.3057e-02],
          [ 9.0782e-02, -1.6318e-01,  2.9001e-02]]],


        [[[ 3.7010e-02, -1.8797e-02, -1.9568e-02],
          [-8.2715e-02,  1.7037e-02,  4.7069e-03],
          [ 5.3735e-02, -3.9003e-04,  1.5702e-02]],

         [[ 1.2933e-01,  1.2755e-01, -1.3224e-01],
          [ 1.7877e-02, -5.8162e-02,  1.5731e-01],
          [-5.9760e-02,  9.6540e-02, -1.0338e-01]],

         [[ 3.8407e-02, -4.9309e-02,  3.8090e-02],
          [-1.8498e-01, -7.6223e-02, -5.0379e-02],
          [ 6.7753e-02, -7.2016e-02,  2.2102e-02]]],


        [[[-1.9581e-03, -6.2443e-02, -1.1872

In [24]:
for param in model.parameters():
    print(param.data.shape)

torch.Size([24, 3, 3, 3])
torch.Size([24])
torch.Size([24])
torch.Size([48, 24, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([36])
torch.Size([36])
torch.Size([48, 36, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48, 48, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([60])
torch.Size([60])
torch.Size([48, 60, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([72])
torch.Size([72])
torch.Size([48, 72, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([84])
torch.Size([84])
torch.Size([48, 84, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([96])
torch.Size([96])
torch.Size([48, 96, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([48, 48, 1, 1])
torch.Size([48])
torch.Size([48])
torch.Size([12, 48, 3, 3])
torch.Size([60])
torch.Size([60])
torch.Size([48, 60, 1, 1]

In [25]:
print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))

Number of model parameters: 176122


### 2.4 Optimizer and Criterion

In [26]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.training.learning_rate, momentum=args.training.momentum, nesterov=True, weight_decay=args.training.weight_decay)

In [27]:
print(f'Criterion: {criterion}\nOptimizer: {optimizer}')

Criterion: CrossEntropyLoss()
Optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0001
)


### 2.5 Training and Validation Loop

In [28]:
start_epoch = 0
best_acc = 0.0

In [29]:
args.training.save_model_path

'toy/experiments/densenet_bc_40_12_baseline'

In [30]:
os.makedirs(args.training.save_model_path, exist_ok=True)

In [34]:
# args.resume_from = False
args.resume_from

'toy/experiment/densenet_bc_40_12_baseline/densenet_bc_40_12.pth'

In [31]:
if args.resume_from:
        print(f"Resuming from checkpoint: {args.resume_from}")
        checkpoint_info = load_checkpoint(args.resume_from, model, optimizer, None)
        start_epoch = checkpoint_info['epoch']
        best_acc = checkpoint_info['best_acc']
        print(f"Resuming from epoch: {start_epoch}, best accuracy: {best_acc*100:.2f}%")

Resuming from checkpoint: toy/experiments/densenet_bc_40_12_baseline/densenet_bc_40_12_cifar10_baseline_best.pth
Resuming from epoch: 244, best accuracy: 9325.00%


In [32]:
initialize_wandb(args)

wandb: Currently logged in as: efradosuryadi (efradosuryadi-universitas-indonesia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [33]:
for epoch in range(start_epoch, args.experiment.num_epochs):
    print('\n')

    # Adjust learning rate
    adjust_learning_rate(optimizer, epoch, args.training.learning_rate)
    
    # Train for one epoch
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, epoch_idx=epoch, args=args, device=device)
    
    # Print the training loss and learning rate
    print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Training Loss: {train_loss:.4f}, Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

    if (epoch + 1) % 1 == 0:
        val_loss, acc = validate(model, val_loader, criterion, device=device)

        wandb.log({
            "Validation Loss": val_loss,
            "Validation Accuracy": acc
        })

        print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            save_checkpoint(f"{args.training.save_model_path}/densenet_bc_40_12_cifar10_baseline_best.pth",model,optimizer,None,epoch,best_acc)
            print(f"Checkpoint saved at epoch {epoch} with accuracy: {best_acc:.2f}%")

print("Training finished.")



Iteration 0: Loss = 0.0208, Top-1 Accuracy = 100.0000, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.0155, Top-1 Accuracy = 99.5668, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.0173, Top-1 Accuracy = 99.5180, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.0170, Top-1 Accuracy = 99.5224, Learning rate = 1.0000e-03
Iteration 400: Loss = 0.0173, Top-1 Accuracy = 99.5129, Learning rate = 1.0000e-03
Iteration 500: Loss = 0.0174, Top-1 Accuracy = 99.5259, Learning rate = 1.0000e-03
Iteration 600: Loss = 0.0176, Top-1 Accuracy = 99.5268, Learning rate = 1.0000e-03
Iteration 700: Loss = 0.0177, Top-1 Accuracy = 99.5275, Learning rate = 1.0000e-03
Epoch [245/300], Training Loss: 0.0176, Learning Rate: 0.001000


100%|██████████| 157/157 [00:03<00:00, 42.72it/s]


Epoch [245/300], Validation Loss: 0.2768, Validation Accuracy: 93.11%


Iteration 0: Loss = 0.0216, Top-1 Accuracy = 100.0000, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.0157, Top-1 Accuracy = 99.6906, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.0168, Top-1 Accuracy = 99.6502, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.0175, Top-1 Accuracy = 99.6211, Learning rate = 1.0000e-03
Iteration 400: Loss = 0.0178, Top-1 Accuracy = 99.5909, Learning rate = 1.0000e-03
Iteration 500: Loss = 0.0176, Top-1 Accuracy = 99.6039, Learning rate = 1.0000e-03
Iteration 600: Loss = 0.0179, Top-1 Accuracy = 99.5840, Learning rate = 1.0000e-03
Iteration 700: Loss = 0.0181, Top-1 Accuracy = 99.5676, Learning rate = 1.0000e-03
Epoch [246/300], Training Loss: 0.0177, Learning Rate: 0.001000


100%|██████████| 157/157 [00:03<00:00, 44.18it/s]


Epoch [246/300], Validation Loss: 0.2800, Validation Accuracy: 93.09%


Iteration 0: Loss = 0.0153, Top-1 Accuracy = 100.0000, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.0147, Top-1 Accuracy = 99.6906, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.0151, Top-1 Accuracy = 99.6502, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.0153, Top-1 Accuracy = 99.6833, Learning rate = 1.0000e-03
Iteration 400: Loss = 0.0167, Top-1 Accuracy = 99.6065, Learning rate = 1.0000e-03
Iteration 500: Loss = 0.0170, Top-1 Accuracy = 99.5665, Learning rate = 1.0000e-03
Iteration 600: Loss = 0.0170, Top-1 Accuracy = 99.5814, Learning rate = 1.0000e-03
Iteration 700: Loss = 0.0167, Top-1 Accuracy = 99.6032, Learning rate = 1.0000e-03
Epoch [247/300], Training Loss: 0.0167, Learning Rate: 0.001000


...

Epoch [298/300], Validation Loss: 0.2916, Validation Accuracy: 93.03%


Iteration 0: Loss = 0.0257, Top-1 Accuracy = 100.0000, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.0117, Top-1 Accuracy = 99.7679, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.0109, Top-1 Accuracy = 99.7979, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.0113, Top-1 Accuracy = 99.7664, Learning rate = 1.0000e-03
Iteration 400: Loss = 0.0113, Top-1 Accuracy = 99.7662, Learning rate = 1.0000e-03
Iteration 500: Loss = 0.0116, Top-1 Accuracy = 99.7661, Learning rate = 1.0000e-03
Iteration 600: Loss = 0.0117, Top-1 Accuracy = 99.7712, Learning rate = 1.0000e-03
Iteration 700: Loss = 0.0119, Top-1 Accuracy = 99.7615, Learning rate = 1.0000e-03
Epoch [299/300], Training Loss: 0.0120, Learning Rate: 0.001000


100%|██████████| 157/157 [00:03<00:00, 43.70it/s]


Epoch [299/300], Validation Loss: 0.2915, Validation Accuracy: 93.13%


Iteration 0: Loss = 0.0128, Top-1 Accuracy = 100.0000, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.0126, Top-1 Accuracy = 99.6597, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.0117, Top-1 Accuracy = 99.7279, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.0120, Top-1 Accuracy = 99.7197, Learning rate = 1.0000e-03
Iteration 400: Loss = 0.0115, Top-1 Accuracy = 99.7506, Learning rate = 1.0000e-03
Iteration 500: Loss = 0.0114, Top-1 Accuracy = 99.7536, Learning rate = 1.0000e-03
Iteration 600: Loss = 0.0115, Top-1 Accuracy = 99.7530, Learning rate = 1.0000e-03
Iteration 700: Loss = 0.0114, Top-1 Accuracy = 99.7526, Learning rate = 1.0000e-03
Epoch [300/300], Training Loss: 0.0115, Learning Rate: 0.001000


100%|██████████| 157/157 [00:03<00:00, 44.05it/s]

Epoch [300/300], Validation Loss: 0.2934, Validation Accuracy: 93.04%
Training finished.


In [34]:
optimizer.param_groups[0]['lr']

0.001

In [35]:
wandb.finish()

Training Accuracy,▆▆▇▇▇▇▇▁▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇█▇▇▇██▇████
Training Loss,██▃█▃▇▇▇▅▆▆▇▆▆▇▅▆▆▆▅▅▅▅▆▆▅▅▇▅▄▅▅▅▆▅▅▁▅▁▅
Validation Accuracy,▄▃▂▄▄▂▁▅▃▆▃▃▆▁▄▆▄▃▆▄▅▆▅▇▄▅▆▆▅▄▇▇█▅▅▃▃▄▂▂
Validation Loss,▁▂▁▂▃▃▂▅▄▃▂▄▆▃▄▄▅▃▄▅▅▅▄▄▄▃▄█▅▅▄▅▄▆▇▆▆▆▆▆
Training Accuracy,99.75259
Training Loss,0.01143
Validation Accuracy,93.04
Validation Loss,0.29341


## 3. Testing

In [36]:
saved_checkpoint_path = args.training.save_model_path + '/densenet_bc_40_12_cifar10_baseline_best.pth'
saved_checkpoint_path

'toy/experiments/densenet_bc_40_12_baseline/densenet_bc_40_12_cifar10_baseline_best.pth'

In [37]:
model_test = create_model(args.model.type,
                          layers=args.model.layers,
                          growth=args.model.growth,
                          compression=args.model.compression,
                          bottleneck=args.model.bottleneck,
                          drop_rate=args.model.drop_rate).to(device)

In [38]:
checkpoint = torch.load(saved_checkpoint_path, map_location=device)
model_test.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [43]:
val_loss, acc = validate(model, val_loader, criterion, device=device)
print(f"Test on DenseNet-BC-40-12, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc:.2f}%")

100%|██████████| 157/157 [00:03<00:00, 44.20it/s]

Test on DenseNet-BC-40-12, Validation Loss: 0.2934, Validation Accuracy: 93.04%
